### Calculate $\delta R$ matrix from combinations (Sequential Version)

Steps are:
- Take $\phi$ and $\eta$ data arrays,starts and stops. 
- Get combinations indices in same manner as done earlier
- For each pair, calculate deltaR as $$\delta r = \sqrt{\delta \eta^{2} + \delta \phi^{2}}$$ where $$\delta \eta = \eta_{2} - \eta_{1}$$$$ \delta \phi = \phi_{2} - \phi_{1}$$
- Get the matrix $$\delta R = \{ \delta r_{ij}\} \quad \forall \: (i,j) \: \in \: combinations \: indices$$


In [15]:
import numpy
import pycuda.driver as cuda
from pycuda.compiler import *
import pycuda.autoinit
import math

In [16]:
# Adapted from gen-reco script by Jim

NUMEVENTS = 100      # exact number of events
AVENUMJETS = 10      # average number of jets per event
PHILOW = -numpy.pi   # bounding box of phi (azimuthal angle) and eta (~polar angle)
PHIHIGH = numpy.pi
ETALOW = -5
ETAHIGH = 5
ERRPHI = 0.01        # detector resolution
ERRETA = 0.01
RECOPROB = 0.95      # probability of not reconstructing a real jet
AVENUMFAKES = 1      # average number of spurious (fake) recontstructions

# simulate the generator-level jets
numgenjets = numpy.random.poisson(AVENUMJETS, NUMEVENTS).astype(numpy.int32)
genstops = numpy.cumsum(numgenjets).astype(numpy.int32)
genstarts = numpy.empty_like(genstops)
genstarts[0] = 0
genstarts[1:] = genstops[:-1]
genphi = numpy.random.uniform(PHILOW, PHIHIGH, genstops[-1]).astype(numpy.float32)
geneta = numpy.random.uniform(ETALOW, ETAHIGH, genstops[-1]).astype(numpy.float32)

# simulate mismeasurement (error in reconstructing phi and eta)
phiwitherr = genphi + numpy.random.normal(0, ERRPHI, genstops[-1]).astype(numpy.float32)
etawitherr = geneta + numpy.random.normal(0, ERRETA, genstops[-1]).astype(numpy.float32)

In [17]:
# Counts per event
lengths = genstops-genstarts

In [18]:
# Cumulative number of possible combinations per event
pairs_len = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_len[1:] = numpy.cumsum(lengths*lengths)

In [19]:
# Initialize deltaR matrix
deltar = numpy.empty(pairs_len[-1], dtype=numpy.float32)

In [20]:
# Calculations. First define deltaeta and deltaphi
def deltaeta(eta1, eta2):
    return eta1-eta2

def deltaphi(phi1, phi2):
    return phi1-phi2

# Okay, now that we are set up, let's calculate deltaR
for i in range(NUMEVENTS):
    pairs_i = pairs_len[i]
    for j in range(lengths[i]):
        for k in range(lengths[i]):
            deltar[pairs_i] = math.hypot(deltaeta(geneta[j], etawitherr[k]), deltaphi(genphi[j], phiwitherr[k]))
            pairs_i +=1

In [22]:
# Print the values
for i in range(6):
    print("Event:{} \n deltaR = {}\n".format(i, deltar[pairs_len[i]:pairs_len[i+1]]))

Event:0 
 deltaR = [  6.51240908e-03   5.56796885e+00   6.79026079e+00   4.17142344e+00
   5.04445362e+00   1.57591176e+00   5.71931410e+00   4.39114809e+00
   3.14141440e+00   5.90915775e+00   5.56314087e+00   1.62661746e-02
   2.26499200e+00   4.12272549e+00   3.48706913e+00   6.19853354e+00
   3.39194703e+00   4.64425039e+00   5.40066910e+00   1.96366191e+00
   6.79249001e+00   2.27630353e+00   1.11183953e-02   6.31839418e+00
   2.68729353e+00   7.79125834e+00   5.64683771e+00   6.85060120e+00
   7.40342140e+00   4.14243126e+00   4.16902733e+00   4.11957073e+00
   6.31287575e+00   6.39884640e-03   6.28871822e+00   3.62221861e+00
   1.88617313e+00   5.41094720e-01   1.77304387e+00   2.97646856e+00
   5.04884100e+00   3.50795269e+00   2.70260191e+00   6.29642534e+00
   5.75590739e-03   6.39696169e+00   6.42941856e+00   6.81431913e+00
   6.74328375e+00   5.32192564e+00   1.57447636e+00   6.19492865e+00
   7.78411341e+00   3.61331105e+00   6.39170837e+00   1.42589891e-02
   5.42367887e+